In [1]:
!pip install praw
!pip install -q langchain openai chromadb
!pip install tiktoken
!pip install openai==0.28.1
!pip install autogen
!pip install pyautogen~=0.2.0b4 docker
!pip install fuzzywuzzy
!pip install --upgrade typing_extensions

  Attempting uninstall: openai
    Found existing installation: openai 1.7.2
    Uninstalling openai-1.7.2:
      Successfully uninstalled openai-1.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=929f7fdeaa13c66a8cf687ad23245436a057cff9d4aa3e8abccb5ffb222ec83e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning)
import praw
from fuzzywuzzy import fuzz, process
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import pandas as pd
import openai
import os
import sys

In [3]:
import os
os.environ["OPENAI_API_KEY"] = 's2'

from openai import OpenAI
client = OpenAI()

def get_completion_from_message(messages, model="gpt-3.5-turbo", temperature=0 ):
  response =client.chat.completions.create(
        model=model,
        messages=messages,
        #temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
  return response.choices[0].message.content

In [6]:
def get_posts(kayword):
  reddit = praw.Reddit(
    client_id="ObBuaI6C5Uli56DiWQltKw",
    client_secret="gHt6ml6pg1MVYX-kzHjr45ltNz2UuA",
    redirect_uri="http://localhost:8080",
    user_agent="python:StockScraper:v1.0 (by /u/Asleep-Pause2513)",
    heck_for_async=False

)



# Specify the subreddit and initial limit
  subreddit_name = kayword
  limit = 1000  # The maximum limit per request

  # Initialize an empty list to store submissions
  all_submissions = []

  # Initial request
  submissions = reddit.subreddit(subreddit_name).new(limit=100000)

  # Append the submissions to the list
  all_submissions.extend(submissions)

  # Fetch more data using the 'after' parameter
  while True:
      try:
          # Get the last submission in the current batch
          last_submission = submissions[-1]

          # Make the next request using 'after' parameter to paginate
          submissions = reddit.subreddit(subreddit_name).new(limit=limit, params={"after": last_submission.name})

          # Check if there are more submissions, and append them
          if not submissions:
              break
          all_submissions.extend(submissions)

      except Exception as e:
          print("An error occurred:", str(e))
          break

  post_titles = []
  post_urls = []
  post_scores = []
  post_comments_nums=[]
  post_comments_texts = []
  post_flair=[]
  comments=[]
  post_authors=[]
  post_texts=[]
  comments_score=[]
  for i in all_submissions :
    post_urls.append(i.url)
    post_scores.append(i.score)
    post_titles.append(i.title)
    post_flair.append(i.link_flair_text)
    post_comments_nums.append(i.num_comments)
    post_authors.append(i.author.name if i.author else 'Deleted')

    # Ensure that we don't get "MoreComments" objects which are placeholders
    i.comments.replace_more(limit=0)

    # Sort the comments by score in descending order and get the top 5
    top_comments = sorted(i.comments.list(), key=lambda comment: comment.score, reverse=True)[:5]
    # Now, extract the data for the top 5 comments
    top_comments_texts = [comment.body for comment in top_comments]
    top_comments_scores = [comment.score for comment in top_comments]

    comments_score.append(top_comments_scores)
    comments.append(top_comments_texts)

    post_texts.append(i.selftext)  # Fetching the full text of the post
  df = pd.DataFrame({
    'Title': post_titles,
    'URL': post_urls,
    'Score': post_scores,
    'Author': post_authors,
    "comments":post_comments_nums,
    'Text': post_texts,
    "post_flair" : post_flair,
    "top comments" : comments,
    "comments score" :comments_score


})
  return df

In [7]:
df1=get_posts("investing")
df2=get_posts("Trading")
df4=get_posts("Cryptocurrencies")
df3=get_posts("stocks")


In [25]:
df1['category'] ="investing"
df2['category'] = "trading"
df3['category'] = "stocks"
df4['category'] = "Cryptocurrencies"

In [26]:
combined_df = pd.concat([df1, df2, df3 , df4], axis=0)
combined_df.shape

(3009, 10)

In [ ]:
df5=get_posts("mutualfunds")
df5["category"] ="mutual funds"

In [32]:
combined_df = pd.concat([df1, df2, df3 , df4 , df5], axis=0)
combined_df.shape

combined_df.to_csv('reddit_data', index=False)


(3872, 10)

In [33]:
combined_df.to_csv('reddit_data', index=False)

In [ ]:
df6=get_posts("StockMarket")
df6["category"] ="StockMarket"

In [36]:
df6["category"] ="stocks"

In [40]:
combined_df = pd.concat([df1, df2, df3 , df4 , df5], axis=0)


combined_df.to_csv('reddit_data', index=False)

In [41]:
combined_df.shape

(3872, 10)

In [ ]:


df7=get_posts("InvestmentClub")
df7["category"] ="investing"



In [45]:
combined_df = pd.concat([df1, df2, df3 , df4 , df5 ,df6], axis=0)


combined_df.to_csv('reddit_data', index=False)

In [46]:
combined_df.shape

(4658, 10)

In [ ]:
df8=get_posts("StocksAndTrading")
df8["category"] ="StocksAndTrading"

In [61]:
df8.shape
combined_df = pd.concat([df1, df2, df3 , df4 , df5 ,df6 , df8,df7], axis=0)


combined_df.to_csv('reddit_data.csv', index=False)

In [62]:
combined_df.shape

(5683, 10)

In [64]:
combined_df = pd.concat([df1, df2, df3 , df4 , df5 ,df6 , df8 , df7 ], axis=0)


combined_df.to_csv('reddit_data', index=False)

In [65]:
import pandas as pd
dataset= pd.read_csv("/content/reddit_data")